### LLM Based Validation Notebook

This notebook has been a helpful tool to help validate the generated QA dataset in addition to the human validation. 

In this notebook, we utilize an LLM to check the quality of the generated questions and answers. The outputs have then been manually reviewed to assess whether the feedback should be incorporated into the dataset.

Accompanying this notebook are multiple text files that contain previous feedback from the LLM. 

In [1]:
import os
import pandas as pd
from tqdm import tqdm
from sqa_system.core.language_model.llm_provider import LLMProvider
from sqa_system.core.data.file_path_manager import FilePathManager
from sqa_system.core.config.models import LLMConfig
llm_config_classification = LLMConfig.from_dict({
    "additional_params": {},
    "endpoint": "OpenAI",
    "name_model": "gpt-4.1",
    "temperature": 0.0,
    "max_tokens": -1
})
llm_adapter = LLMProvider().get_llm_adapter(llm_config_classification)


fpm = FilePathManager()
current_dir = os.getcwd()
deep_dataset_location = fpm.combine_paths(fpm.get_parent_directory(current_dir), "qa_datasets", "full", "deep_graph_dataset.csv")
reduced_deep_dataset_location = fpm.combine_paths(fpm.get_parent_directory(current_dir), "qa_datasets", "reduced", "reduced_deep_graph_dataset.csv")
reduced_deep_dataset_df = pd.read_csv(reduced_deep_dataset_location)
deep_dataset_df = pd.read_csv(deep_dataset_location)

Rotating log file
2025-04-19 16:01:34,924 - New session started


In [2]:
def validate_grammar(text: str) -> tuple[bool, str]:
    """Checks whether the string is grammatically correct."""
    prompt = f"You are a grammar checker. Check the following text for grammatical correctness:\n\n{text}\n\nIf the text has grammatical issues say: 'The text has grammatical issues. Here is the corrected text:' and provide the corrected text. If the text is grammatically correct, say: 'The text is grammatically correct.' Do not add any additional details or explanations. Do not add quotation marks (single or double) to any words or phrases unless they are already present in the original text. Maintain the original quotation marks."
    response = llm_adapter.generate(prompt).content
    if "The text is grammatically correct." in response:
        return True, text
    else:
        corrected_text = response.split("Here is the corrected text:")[-1].strip()
        return False, corrected_text
    
def validate_question_answer_consistency(question: str, answer: str) -> tuple[bool, str]:
    """Checks whether the answer is consistent with the question."""
    prompt = f"""You are an answer validator. Check whether the following answer is consistent with the question by following the checklist: \n
    1. Does the answer directly address the question?
    2. Does the answer add any additional details that the question did not ask for?
    3. Are the verbs used in the question and answer consistent?
    4. Are the nouns used in the question and answer consistent?  

    Note that the answer should repeat the question in some way for example: 'What is the research object of the paper with the title X?' should be answered with 'The research object of the paper with the title X is Y.' Therefore, the answer should not just simply state the facts.
    
    If any of the checklist items are not satisfied, say: 'Checklist item(s) X,Y is/are not satisfied. Here is the corrected answer:' and provide the corrected answer where each of the checklist items are satisfied. If all checklist items are satisfied, say: 'The answer is consistent with the question. Do not add any additional details or explanations. '\n\nQuestion: {question}\nAnswer: {answer}"""
    response = llm_adapter.generate(prompt).content
    if "The answer is consistent with the question" in response:
        return True, answer   
    else:
        corrected_answer = response.split("Here is the corrected answer:")[-1].strip()
        return False, corrected_answer
    
def validate_typed_question(question_template: str):
    """Checks whether the question template correctly adds the type of the placeholder in the question."""
    prompt = f"""You are a question validator. You receive a question with a placeholder that is indicated as '[placeholder]'. For any given placeholder, there is a typed string which needs to be contained within the question (not the placeholder) to indicate the type of constraint that the question is asking for. You need to check whether the question correctly contains the typed string that is expected for the given placeholder. The table of placeholders and their expected typed strings is as follows:
        
    | Placeholder | Typed String | 
    | ----------- | ----------- |
    | -           | doi         |
    | paper title | paper with the title|
    | -           | author     |
    | -           | venue       |
    | -           | research field |
    | year        | publication year |
    | research level | research level |
    | paper class name | paper class |
    | Threat to Validity | threat to validity |
    | -           | input data  | 
    | -           | replication package link | 
    | -           | tool support |
    | -           | threats to validity guideline |
    | research object name | research object |
    | evaluation method name | evaluation method |
    | -           | evaluation guideline |
    | property name | property |
    | sub-property name | property|
    | Provides Replication Package | provides replication package |
    
    If the question correctly includes the typed string that is expected for the given placeholder in the question, say 'The question correctly includes the typed string that is expected for the given placeholder.' If the question does not include the typed string that is expected for the given placeholder, say 'The question does not include the typed string that is expected for the given placeholder. Here is the corrected question:' and provide the corrected question. Do not add any additional details or explanations. 
    
    Examples:
    Question: In which venue has the paper with the title '[paper title]' been published?
    Answer: The question correctly includes the typed string that is expected for the given placeholder.
    Question: In which venue has '[paper title]' been published?
    Answer: The question does not include the typed string that is expected for the given placeholder. Here is the corrected question: In which venue has the paper with the title ['paper title'] been published?
    
    
    The Question with placeholder: {question_template}
    """
    
    response = llm_adapter.generate(prompt).content
    if "The question correctly includes the typed string that is expected for the given placeholder." in response:
        return True, question_template
    else:
        corrected_question = response.split("Here is the corrected question:")[-1].strip()
        if corrected_question == question_template:
            return True, question_template        
        return False, corrected_question
    
def validate_correct_verbs(text: str) -> tuple[bool, str]:
    """Validates whether the verbs in the text are correctly used."""
    prompt = f"""You are a verb-validator. You will receive a single-sentence input and must check every verb that governs one of the listed nouns. Matching is case-insensitive and applies to both singular and plural forms. Any inflected form of an allowed verb (e.g. “apply,” “applies,” “applying,” “applied”) is also acceptable.

    Allowed-verbs table:
    ```markdown
    | Noun               | Allowed Verbs          |
    |--------------------|------------------------|
    | research object    | investigate, evaluate  |
    | evaluation method  | evaluate, apply        |
    | threat to validity | discuss                |
    | guideline          | use                    |
    | property           | investigate, evaluate  |
    | input data         | use                    |
    ```
    If the verbs in the text are correctly used, say 'The verbs in the text are correctly used.' If the verbs in the text are not correctly used, say 'The verbs in the text are not correctly used. Here is the corrected text:' and provide the corrected text. Do not add any additional details or explanations.

    Text: Which paper evaluates the property robustness?” 
    Answer: The verbs in the text are not correctly used. Here is the corrected text: Which paper investigates the property robustness?
    
    Text: {text}
    Answer:
    
    {text}
    """

    response = llm_adapter.generate(prompt).content
    if "The verbs in the text are correctly used." in response:
        return True, text
    else:
        corrected_text = response.split("Here is the corrected text:")[-1].strip()
        return False, corrected_text

def validate_question_to_template(question:str, template:str) -> tuple[bool, str]:
    """Validates whether the template is consistent with the question."""
    prompt = f"""You are a question to template validator. You receive a question and a template. You need to check whether the template is consistent with the question. A template is consistent with the question, if the template is the exact same as the question except for the placeholders that are indicated as '[placeholder]'. The placeholders are the only parts of the template that can be different from the question. If the template is consistent with the question, say 'The template is consistent with the question.' If the template is not consistent with the question, say 'The template is not consistent with the question. Here is the corrected template:' and provide the corrected template.\nQuestion: {question}\nTemplate: {template}"""
    response = llm_adapter.generate(prompt).content
    if "The template is consistent with the question." in response:
        return True, template
    else:
        corrected_template = response.split("Here is the corrected template:")[-1].strip()
        return False, corrected_template
    
def get_list_of_difference(first: str, second: str) -> list[str]:
    """Returns a list of those words that are in the second string but not in the first string."""
    first_words = set(first.split())
    second_words = set(second.split())
    difference = second_words - first_words
    return list(difference)

### Validate Reduced QA Dataset

In [ ]:
for index, row in tqdm(reduced_deep_dataset_df.iterrows(), total=len(reduced_deep_dataset_df)):
    question = row['question']
    golden_answer = row['golden_answer']
    question_template = row['updated template']
    is_typed = row['semi-typed']
    
    is_grammatically_correct, corrected_question = validate_grammar(question)
    if not is_grammatically_correct:
        print(f"Line: {index + 2}, Found grammar issues in the question")
        print(f"Original Question: {question}")
        print(f"Corrected Question: {corrected_question}")
        print(f"Difference Question Corrected: {get_list_of_difference(question, corrected_question)}")
        print(f"Difference Question Original: {get_list_of_difference(corrected_question, question)}")
        print()
        
    is_grammatically_correct, corrected_answer = validate_grammar(golden_answer)
    if not is_grammatically_correct:
        print(f"Line: {index + 2}, Found grammar issues in the answer")
        print(f"Original Answer: {golden_answer}")
        print(f"Corrected Answer: {corrected_answer}")
        print(f"Difference Answer Corrected: {get_list_of_difference(golden_answer, corrected_answer)}")
        print(f"Difference Answer Original: {get_list_of_difference(corrected_answer, golden_answer)}")
        print()
        
    is_consistent, corrected_answer = validate_question_answer_consistency(corrected_question, corrected_answer)
    if not is_consistent:
        print(f"Line: {index + 2}, Found inconsistency between question and answer")
        print(f"Original Answer: {golden_answer}")
        print(f"Corrected Answer: {corrected_answer}")
        print(f"Difference Answer Corrected: {get_list_of_difference(golden_answer, corrected_answer)}")
        print(f"Difference Answer Original: {get_list_of_difference(corrected_answer, golden_answer)}")
        print()
    
    if is_typed:
        is_valid, corrected_question_template = validate_typed_question(question_template)
        if not is_valid:
            print(f"Line: {index + 2}, found typing issues in the question template")
            print(f"Original Question Template: {question_template}")
            print(f"Corrected Question Template: {corrected_question_template}")
            print(f"Difference Question Template Corrected: {get_list_of_difference(question_template, corrected_question_template)}")
            print(f"Difference Question Template Original: {get_list_of_difference(corrected_question_template, question_template)}")
            print()
            
        is_noun_verb_correct, corrected_question = validate_correct_verbs(corrected_question)
        if not is_noun_verb_correct:
            print(f"Line: {index + 2}, found verb issues in the question")
            print(f"Original Question: {question}")
            print(f"Corrected Question: {corrected_question}")
            print(f"Difference Question Corrected: {get_list_of_difference(question, corrected_question)}")
            print(f"Difference Question Original: {get_list_of_difference(corrected_question, question)}")
            print()

        is_noun_verb_correct, corrected_answer = validate_correct_verbs(corrected_answer)
        if not is_noun_verb_correct:
            print(f"Line: {index + 2}, found verb issues in the answer")
            print(f"Original Answer: {golden_answer}")
            print(f"Corrected Answer: {corrected_answer}")
            print(f"Difference Answer Corrected: {get_list_of_difference(golden_answer, corrected_answer)}")
            print(f"Difference Answer Original: {get_list_of_difference(corrected_answer, golden_answer)}")
            print()

  7%|▋         | 3/44 [00:16<03:35,  5.26s/it]

Line: 5, Found grammar issues in the answer
Original Answer: The publications with the paper class personal experience, ranked in descending order by publication year, are: 1. Data-Centric Communication and Containerization for Future Automotive Software Architectures (2018); 2. Towards a Reference Architecture for Cloud-Based Plant Genotyping and Phenotyping Analysis Frameworks (2017).
Corrected Answer: The publications with the paper class of personal experience, ranked in descending order by publication year, are: 1. Data-Centric Communication and Containerization for Future Automotive Software Architectures (2018); 2. Towards a Reference Architecture for Cloud-Based Plant Genotyping and Phenotyping Analysis Frameworks (2017).
Difference Answer Corrected: ['of']
Difference Answer Original: []



 11%|█▏        | 5/44 [00:27<03:30,  5.41s/it]

Line: 7, Found inconsistency between question and answer
Original Answer: Konstantinos Plakidas has published three papers in the following years: 2021, 2020, and 2019. Therefore, the distribution of papers published per year is as follows: one paper in 2021, one paper in 2020, and one paper in 2019, resulting in a total of three papers over three years, which is one paper per year.
Corrected Answer: The number of papers that the author Konstantinos Plakidas published per publication year is one paper in 2019, one paper in 2020, and one paper in 2021.
Difference Answer Corrected: ['number', 'author', 'The', '2021.', 'that', 'publication']
Difference Answer Original: ['a', 'over', 'as', 'years,', 'Therefore,', 'resulting', 'has', 'years:', 'following', 'year.', 'three', 'follows:', 'total', '2021,', '2019.', 'distribution', 'which']



 16%|█▌        | 7/44 [00:37<03:10,  5.16s/it]

...

 91%|█████████ | 40/44 [03:37<00:22,  5.53s/it]

Line: 42, Found grammar issues in the answer
Original Answer: In the year 2020, there are two papers that investigate the research object Architecture Evolution. In comparison, in the year 2021, there are two papers that investigate the research object.
Corrected Answer: In the year 2020, there are two papers that investigate the research object of Architecture Evolution. In comparison, in the year 2021, there are two papers that investigate the research object.
Difference Answer Corrected: ['of']
Difference Answer Original: []

Line: 42, Found inconsistency between question and answer
Original Answer: In the year 2020, there are two papers that investigate the research object Architecture Evolution. In comparison, in the year 2021, there are two papers that investigate the research object.
Corrected Answer: The number of papers investigating the research object Architecture Evolution in the publication year 2020 is two, whereas the number in the publication year 2021 is also two.
Diff

 95%|█████████▌| 42/44 [03:46<00:10,  5.06s/it]

Line: 44, Found grammar issues in the answer
Original Answer: The paper class that has the most papers applying the evaluation method Benchmark in the publication year 2020 is the proposal of a solution.
Corrected Answer: The paper class that has the most papers applying the evaluation method Benchmark in the publication year 2020 is the proposal of a solution class.
Difference Answer Corrected: ['solution', 'class.']
Difference Answer Original: ['solution.']

Line: 44, found verb issues in the question
Original Question: Among papers that apply the evaluation method Benchmark, what is the paper class that is most frequently applied to papers published in the publication year 2020?
Corrected Question: Among papers that apply the evaluation method Benchmark, what is the paper class that is most frequently evaluated in papers published in the publication year 2020?
Difference Question Corrected: ['evaluated']
Difference Question Original: ['to', 'applied']



100%|██████████| 44/44 [03:56<00:00,  5.38s/it]


In [5]:
for index, row in tqdm(reduced_deep_dataset_df.iterrows(), total=len(reduced_deep_dataset_df)):
    question = row['question']
    question_template = row['updated template']
    template_consistent, corrected_template = validate_question_to_template(question, question_template)
    if not template_consistent:
        print(f"Line: {index + 2}, found template issues in the question")
        print(f"Original Question Template: {question_template}")
        print(f"Corrected Question Template: {corrected_template}")
        print(f"Difference Question Template Corrected: {get_list_of_difference(question_template, corrected_template)}")
        print(f"Difference Question Template Original: {get_list_of_difference(corrected_template, question_template)}")
        print()

 18%|█▊        | 8/44 [00:06<00:28,  1.24it/s]

Line: 9, found template issues in the question
Original Question Template: What are the evaluation methods used in the paper with the title '[paper title]'?
Corrected Question Template: What are the evaluation methods applied in the paper with the title '[paper title]'?
Difference Question Template Corrected: ['applied']
Difference Question Template Original: ['used']



 25%|██▌       | 11/44 [00:08<00:28,  1.18it/s]

Line: 12, found template issues in the question
Original Question Template: What are the evaluation methods used by the paper with the title '[paper title]' compared to the methods applied in the paper with the title '[paper title]'?
Corrected Question Template: What are the evaluation methods used by the paper with the title '[paper title]' compared to the evaluation methods applied in the paper with the title '[paper title]'?
Difference Question Template Corrected: []
Difference Question Template Original: []



 30%|██▉       | 13/44 [00:09<00:22,  1.35it/s]

...

 70%|███████   | 31/44 [00:24<00:10,  1.28it/s]

Line: 32, found template issues in the question
Original Question Template: How many different evaluation methods did the author [author name] use in papers that evaluate the property [property name]?
Corrected Question Template: How many different evaluation methods did the author [author name] apply in papers that evaluate the property [property name]?
Difference Question Template Corrected: ['apply']
Difference Question Template Original: ['use']



 95%|█████████▌| 42/44 [00:32<00:01,  1.41it/s]

Line: 43, found template issues in the question
Original Question Template: Which papers published in the publication year [year] investigate the research object [research object name] and have not used input data in their work?
Corrected Question Template: Which papers published in the publication year [year] investigate the research object [research object name] and have not used input data?
Difference Question Template Corrected: ['data?']
Difference Question Template Original: ['their', 'work?', 'data']



100%|██████████| 44/44 [00:33<00:00,  1.31it/s]


### Validate Full Dataset

In [4]:
for index, row in tqdm(deep_dataset_df.iterrows(), total=len(deep_dataset_df)):
    question = row['question']
    golden_answer = row['golden_answer']
    question_template = row['updated template']
    is_typed = row['semi-typed']
    
    is_grammatically_correct, corrected_question = validate_grammar(question)
    if not is_grammatically_correct:
        print(f"Line: {index + 2}, Found grammar issues in the question")
        print(f"Original Question: {question}")
        print(f"Corrected Question: {corrected_question}")
        print(f"Difference Question Corrected: {get_list_of_difference(question, corrected_question)}")
        print(f"Difference Question Original: {get_list_of_difference(corrected_question, question)}")
        print()
        
    is_grammatically_correct, corrected_answer = validate_grammar(golden_answer)
    if not is_grammatically_correct:
        print(f"Line: {index + 2}, Found grammar issues in the answer")
        print(f"Original Answer: {golden_answer}")
        print(f"Corrected Answer: {corrected_answer}")
        print(f"Difference Answer Corrected: {get_list_of_difference(golden_answer, corrected_answer)}")
        print(f"Difference Answer Original: {get_list_of_difference(corrected_answer, golden_answer)}")
        print()
        
    is_consistent, corrected_answer = validate_question_answer_consistency(corrected_question, corrected_answer)
    if not is_consistent:
        print(f"Line: {index + 2}, Found inconsistency between question and answer")
        print(f"Original Answer: {golden_answer}")
        print(f"Corrected Answer: {corrected_answer}")
        print(f"Difference Answer Corrected: {get_list_of_difference(golden_answer, corrected_answer)}")
        print(f"Difference Answer Original: {get_list_of_difference(corrected_answer, golden_answer)}")
        print()
    
    if is_typed:
        is_valid, corrected_question_template = validate_typed_question(question_template)
        if not is_valid:
            print(f"Line: {index + 2}, found typing issues in the question template")
            print(f"Original Question Template: {question_template}")
            print(f"Corrected Question Template: {corrected_question_template}")
            print(f"Difference Question Template Corrected: {get_list_of_difference(question_template, corrected_question_template)}")
            print(f"Difference Question Template Original: {get_list_of_difference(corrected_question_template, question_template)}")
            print()
        
        is_noun_verb_correct, corrected_question = validate_correct_verbs(corrected_question)
        if not is_noun_verb_correct:
            print(f"Line: {index + 2}, found verb issues in the question")
            print(f"Original Question: {question}")
            print(f"Corrected Question: {corrected_question}")
            print(f"Difference Question Corrected: {get_list_of_difference(question, corrected_question)}")
            print(f"Difference Question Original: {get_list_of_difference(corrected_question, question)}")
            print()

        is_noun_verb_correct, corrected_answer = validate_correct_verbs(corrected_answer)
        if not is_noun_verb_correct:
            print(f"Line: {index + 2}, found verb issues in the answer")
            print(f"Original Answer: {golden_answer}")
            print(f"Corrected Answer: {corrected_answer}")
            print(f"Difference Answer Corrected: {get_list_of_difference(golden_answer, corrected_answer)}")
            print(f"Difference Answer Original: {get_list_of_difference(corrected_answer, golden_answer)}")
            print()

  4%|▎         | 6/170 [00:16<08:15,  3.02s/it]

Line: 8, Found grammar issues in the question
Original Question: Which papers have the research level secondary research?
Corrected Question: Which papers have the research level of secondary research?
Difference Question Corrected: ['of']
Difference Question Original: []



  6%|▋         | 11/170 [00:31<06:40,  2.52s/it]

Line: 13, Found grammar issues in the question
Original Question: How many papers have the paper class personal experience paper?
Corrected Question: How many papers have the paper class "personal experience paper"?
Difference Question Corrected: ['paper"?', '"personal']
Difference Question Original: ['paper?', 'personal']

Line: 13, Found grammar issues in the answer
Original Answer: There are two publications that have the paper class personal experience paper.
Corrected Answer: There are two publications that have the paper class of personal experience paper.
Difference Answer Corrected: ['of']
Difference Answer Original: []

Line: 13, Found inconsistency between question and answer
Original Answer: There are two publications that have the paper class personal experience paper.
Corrected Answer: The number of papers that have the paper class "personal experience paper" is two.
Difference Answer Corrected: ['"personal', 'two.', 'is', 'of', 'paper"', 'The', 'papers', 'number']
Differe

  7%|▋         | 12/170 [00:33<06:36,  2.51s/it]

...

 98%|█████████▊| 166/170 [09:03<00:11,  2.98s/it]

Line: 168, Found inconsistency between question and answer
Original Answer: The distribution of papers investigating the research object Architecture Extraction between 2018 and 2020 is as follows: two papers were published in 2018, one paper was published in 2019, and one paper was published in 2020.
Corrected Answer: The papers that investigate the research object Architecture Extraction are distributed between the publication years 2018 and 2020 as follows: two papers were published in 2018, one paper was published in 2019, and one paper was published in 2020.
Difference Answer Corrected: ['publication', 'investigate', 'years', 'are', 'that', 'distributed']
Difference Answer Original: ['distribution', 'investigating', 'of', 'is']



 99%|█████████▉| 168/170 [09:10<00:06,  3.13s/it]

Line: 170, Found inconsistency between question and answer
Original Answer: The distribution of papers with the evaluation method Argumentation between 2019 and 2021 is as follows: In 2019 two papers were published and in 2020 three papers were published. However, in 2021 no paper was published with the evaluation method Argumentation.
Corrected Answer: The distribution of papers with the evaluation method Argumentation between the publication years 2019 and 2021 is: in 2019, two papers; in 2020, three papers; and in 2021, no papers.
Difference Answer Corrected: ['publication', 'years', '2020,', 'is:', 'papers;', '2019,', 'papers.', '2021,']
Difference Answer Original: ['was', '2020', 'follows:', 'In', 'is', 'were', 'Argumentation.', 'paper', 'published.', 'However,', 'published', 'as']



100%|██████████| 170/170 [09:18<00:00,  3.28s/it]


In [3]:
for index, row in tqdm(deep_dataset_df.iterrows(), total=len(deep_dataset_df)):
    question = row['question']
    question_template = row['updated template']
    template_consistent, corrected_template = validate_question_to_template(question, question_template)
    if not template_consistent:
        print(f"Line: {index + 2}, found template issues in the question")
        print(f"Original Question Template: {question_template}")
        print(f"Corrected Question Template: {corrected_template}")
        print(f"Difference: {get_list_of_difference(question_template, corrected_template)}")
        print()

  0%|          | 0/170 [00:00<?, ?it/s]Warning: Traces will not be logged. Call weave.init to log your traces to a project.
 (subsequent messages of this type will be suppressed)
 29%|██▉       | 50/170 [00:37<01:42,  1.17it/s]

Line: 51, found template issues in the question
Original Question Template: Which paper applies [evaluation method name] as a method in their evaluation?
Corrected Question Template: Which paper applies [placeholder] as a method in their evaluation?
Difference: ['[placeholder]']



 30%|███       | 51/170 [00:38<01:37,  1.21it/s]

Line: 52, found template issues in the question
Original Question Template: In which paper is the [property name] evaluated?
Corrected Question Template: In which paper is the [placeholder] evaluated?
Difference: ['[placeholder]']



 32%|███▏      | 54/170 [00:41<01:36,  1.20it/s]

...

 98%|█████████▊| 166/170 [01:59<00:03,  1.16it/s]

Line: 167, found template issues in the question
Original Question Template: Which paper class was used the most for publications that investigate [research object name]s in [year]?
Corrected Question Template: Which paper class was used the most for publications that investigated [research object name]s in [year]?
Difference: ['investigated']



 99%|█████████▉| 168/170 [02:01<00:01,  1.17it/s]

Line: 169, found template issues in the question
Original Question Template: What is the distribution of papers that investigate the object [research object name] between [year] and [year]?
Corrected Question Template: What is the distribution of papers that investigate the object [architecture description] between [2018] and [2020]?
Difference: ['[architecture', '[2018]', 'description]', '[2020]?']



100%|██████████| 170/170 [02:02<00:00,  1.39it/s]
